<a href="https://colab.research.google.com/github/Hushpuppyzac/DLI-Assignment/blob/main/Eric_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import requests
import nbformat
from IPython import get_ipython

def run_notebook_from_github(url):
    """
    Downloads and executes a Jupyter notebook from a GitHub raw URL.

    Parameters:
    url (str): Raw GitHub URL to a .ipynb notebook file
    """
    print("🔄 Downloading notebook from GitHub...")
    try:
        response = requests.get(url)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to download notebook: {e}")
        return

    print("📖 Parsing notebook content...")
    try:
        notebook = nbformat.reads(response.text, as_version=4)
    except Exception as e:
        print(f"❌ Failed to parse notebook: {e}")
        return

    ipython = get_ipython()
    print("⚙️  Running notebook cells...\n")

    for i, cell in enumerate(notebook.cells):
        if cell.cell_type == 'code':
            try:
                print(f"▶️  Executing cell [{i + 1}]...")
                ipython.run_cell(cell.source)
            except Exception as e:
                print(f"❌ Error in cell [{i + 1}]: {e}")

    print("\n✅ All executable cells have been processed.")

# 🔗 Your GitHub raw notebook URL
notebook_url = "https://raw.githubusercontent.com/Hushpuppyzac/DLI-Assignment/main/CleanedData.ipynb"

# ▶️ Run it
run_notebook_from_github(notebook_url)

🔄 Downloading notebook from GitHub...
📖 Parsing notebook content...
⚙️  Running notebook cells...

▶️  Executing cell [2]...


/tmp/ipython-input-338159769.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].apply(lambda x: 1 if x != 'BENIGN' else 0)


 AFTER CLEANING
Total rows after cleaning: 223108
Label distribution after cleaning:
Label
1    128016
0     95092
Name: count, dtype: int64
Class Distribution Before Balancing:
Label
1    128016
0     95092
Name: count, dtype: int64

 AFTER UNDERSAMPLING
Total rows after balancing: 190184
Class balance:
Label
1    95092
0    95092
Name: count, dtype: int64

SAMPLE OF CLEANED DATAFRAME
Total Rows     : 223108
DDoS Attacks   : 128016
Benign Records : 95092

 Cleaned DataFrame (First 5 Rows):


,Destination Port,Flow Duration,Label
0,54865,3,0
1,55054,109,0
2,55055,52,0
3,46236,34,0
4,54863,3,0



 DDoS Samples (First 5):


...


 Benign Samples (First 5):


,Destination Port,Flow Duration,Label
0,54865,3,0
1,55054,109,0
2,55055,52,0
3,46236,34,0
4,54863,3,0



 Sample Extracted Features (First 5):


,Flow Preview,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
0,54865 | 3,0,1.0,1200000.0,1.5,1.5
1,55054 | 109,0,1.0,1.0,54.5,1.5
2,55055 | 52,0,1.0,1.0,26.0,1.5
3,46236 | 34,0,1.0,1.0,17.0,1.5
4,54863 | 3,0,1.0,1200000.0,1.5,1.5



 Final Columns:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd 

Algorithm : SVM
Confusion Matrix Heatmap

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

Check dataset

In [19]:
# If not already loaded from the notebook cleaning step
# df = pd.read_csv('CleanedDataset.csv')  # Replace with actual filename if saved

# Optional: check the first few rows
df.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Idle Std,Idle Max,Idle Min,Label,Flow Preview,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,0.0,0,0,0,54865 | 3,0,1.0,1200000.0,1.5,1.5
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,0.0,0,0,0,55054 | 109,0,1.0,1.0,54.5,1.5
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,0.0,0,0,0,55055 | 52,0,1.0,1.0,26.0,1.5
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,0.0,0,0,0,46236 | 34,0,1.0,1.0,17.0,1.5
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,0.0,0,0,0,54863 | 3,0,1.0,1200000.0,1.5,1.5


✅ Step 2: Prepare the Data for SVM

In [22]:
# Example column selection – adjust based on your actual cleaned data
X = df.drop('Label', axis=1)  # Replace 'Label' with your actual target column
y = df['Label']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)  # This converts categorical labels to numeric

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
# Find all columns with non-numeric types
non_numeric_cols = X.select_dtypes(include=['object']).columns
print("Non-numeric columns:\n", non_numeric_cols)

# Check sample values
for col in non_numeric_cols:
    print(f"\nColumn: {col}")
    print(X[col].unique()[:10])  # Print first 10 unique values for each problematic column


Non-numeric columns:
 Index(['Flow Preview'], dtype='object')

Column: Flow Preview
['54865 | 3' '55054 | 109' '55055 | 52' '46236 | 34' '54863 | 3'
 '54871 | 1022' '54925 | 4' '54925 | 42' '9282 | 4' '55153 | 4']


In [27]:
# Step 1: Split the 'Flow Preview' column into two new columns
X[['Flow_Part1', 'Flow_Part2']] = X['Flow Preview'].str.split('|', expand=True)

# Step 2: Convert them to float (or int)
X['Flow_Part1'] = X['Flow_Part1'].str.strip().astype(float)
X['Flow_Part2'] = X['Flow_Part2'].str.strip().astype(float)

# Step 3: Drop the original column
X.drop('Flow Preview', axis=1, inplace=True)


In [29]:
import numpy as np

# Replace inf and -inf with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Then fill or drop NaN
X.fillna(0, inplace=True)  # or X.dropna(inplace=True) if appropriate


In [30]:
print("Any NaNs left?:", X.isnull().values.any())
print("Any infs left?:", np.isinf(X.values).any())


Any NaNs left?: False
Any infs left?: False


✅ Step 3: Train the SVM Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)

# Predict
y_pred = svm.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
